# Tests

In [ ]:
import importlib
import evaluator
importlib.reload(evaluator)

evaluator.run_deterministic_tests(use_cache=True)
# evaluator.run_probabilistic_tests(use_cache=True)

# Programs

In [70]:
from daphne import daphne
import os, json

def ast_helper(fname,directory):
    sugared_fname = '../prob_prog/hw/hw6/CS532-HW6/{}/{}'.format(directory,fname)
    desugared_ast_json_fname = '/Users/gw/repos/prob_prog/' + sugared_fname.replace('.daphne','.json')
    if os.path.isfile(desugared_ast_json_fname):
        with open(desugared_ast_json_fname) as f:
            ast = json.load(f)
    else:
        #note: the sugared path that goes into daphne desugar should be with respect to the daphne path!
        ast = daphne(['desugar-hoppl-cps', '-i', sugared_fname]) 

        with open(desugared_ast_json_fname, 'w') as f:
            json.dump(ast, f)
    return ast

i=4
fname = '{}.daphne'.format(i)
exp = ast_helper(fname,directory='programs')
%cat programs/4.daphne

(let [mu (sample (normal 1 (sqrt 5)))
           sigma (sqrt 2)
           lik (normal mu sigma)]
       (observe lik 8)
       (observe lik 9)
       mu)

In [ ]:
from evaluator import evaluate, sample_from_prior

In [ ]:
sample_from_prior(exp)

In [ ]:
output = lambda x: x
res =  evaluate(exp, env=None)('addr_start', output)
cont, args, sigma = res
res

In [ ]:
cont, args, sigma = res
res = cont(*args)
res, type(res)


# Importance sampling

In [67]:
%%bash
python importance_sampling.py




Sample of prior of program 1: 329
covariance:  10136.326214381439
posterior mean: [100.27679]



Sample of prior of program 2: 0.7192243337631226
covariance:  0.8215114511659168
posterior mean: [7.228802]



Sample of prior of program 3: tensor([1, 2, 2, 0, 1, 2, 2, 2, 2, 1, 0, 1, 2, 2, 2, 2, 2])
covariance:  [[ 6.23066341e-01  2.00989827e-01 -1.81050433e-02 -1.65233088e-02
  -1.34005430e-03 -2.97007979e-02 -2.54959391e-02 -1.81702169e-02
   1.23129709e-02 -1.84960769e-01 -3.12095056e-02 -3.13152619e-02
   6.20650097e-02  3.61935019e-02 -7.35044954e-04 -4.09876390e-02
   7.94255037e-02]
 [ 2.00989827e-01  6.50417323e-01  4.71642744e-02  4.23439449e-02
   3.13351602e-04 -4.11443381e-02 -6.25577110e-02 -3.90557418e-04
  -1.54502784e-02 -2.26365346e-01 -1.21253991e-02 -8.61383717e-02
   4.97445048e-03  2.43736806e-02  7.62757319e-02 -6.23329955e-02
   8.97188891e-02]
 [-1.81050433e-02  4.71642744e-02  3.14683410e-01  1.43065055e-01
  -3.91723602e-04  3.33573734e-02  1.29129921e-02 -7.1

## develop

In [22]:
from torch import tensor
import torch
from evaluator import evaluate

In [98]:
output = lambda x: x
res =  evaluate(exp, env=None)('addr_start', output)

In [5]:
logW = tensor(0.0)
while type(res) is tuple:
    cont, args, sigma = res
    if sigma['type'] == 'observe':
        distribution = sigma['distribution']
        observed_constant = sigma['observed_constant']
        logW += distribution.log_prob(sigma['observed_constant'])
        
    res = cont(*args)

In [72]:
def get_IS_sample(exp):
    #init calc:
    output = lambda x: x
    res =  evaluate(exp, env=None)('addr_start', output)
    #TODO : hint, "get_sample_from_prior" as a basis for your solution

    logW = tensor(0.0)
    while type(res) is tuple:
        cont, args, sigma = res
        if sigma['type'] == 'observe':
            distribution = sigma['distribution']
            observed_constant = sigma['observed_constant']
            logW += distribution.log_prob(sigma['observed_constant']).item()
        res = cont(*args)
        
    return logW, res

logW, res = get_IS_sample(exp)
logW, res

(tensor(-55.5383, grad_fn=<AddBackward0>), tensor(-1.7842))

In [73]:
num_samples = 10000

log_weights = []
values = []
for i in range(num_samples):
    logW, sample = get_IS_sample(exp)
    log_weights.append(logW)
    values.append(sample)


In [74]:
log_weights = tensor(log_weights)
log_weights

tensor([-62.8138, -33.0696, -33.2654,  ..., -25.6846, -50.9490, -19.4317])

In [35]:
values = torch.stack(values)
values = values.reshape((values.shape[0],values.size().numel()//values.shape[0]))
values

tensor([[ 2.7904],
        [-1.1711],
        [ 0.3792]])

In [52]:
log_Z = torch.logsumexp(log_weights,0) - torch.log(torch.tensor(log_weights.shape[0],dtype=float))
log_Z

tensor(-20.0542, dtype=torch.float64)

In [50]:
torch.logsumexp(tensor([-1.,-1.,-1.]),0) # converts log_probs to probs, then sums, then logs https://pytorch.org/docs/stable/generated/torch.logsumexp.html
# the product sum of the probs in log space. ie joint prob of iid pieces



tensor(0.0986)

In [54]:
log_norm_weights = log_weights - log_Z
weights = torch.exp(log_norm_weights).detach().numpy()
weighted_samples = (torch.exp(log_norm_weights).reshape((-1,1))*values.float()).detach().numpy()
weighted_samples

array([[ 8.3713150e+00],
       [-2.0643113e-13],
       [ 6.5225954e-08]], dtype=float32)

# SMC

In [152]:
import smc
import importlib
importlib.reload(smc)

<module 'smc' from '/Users/gw/repos/prob_prog/hw/hw6/CS532-HW6/smc.py'>

In [ ]:
%%time
logZ, particles = smc.SMC(100000,exp)

In SMC step 0, Zs:  []


In [ ]:
samples_array = np.array([sample.item() for sample in particles])
samples_array.mean(), samples_array.var()

## develop

In [89]:
import numpy as np

array([3215, 7634, 1280, ...,  188, 6755, 5700])

In [90]:
n_particles = log_weights.size().numel()
log_weights = tensor(log_weights)
log_Z = torch.logsumexp(log_weights,0) #- torch.log(torch.tensor(log_weights.shape[0],dtype=float)) # second piece normalizes to num_samples, instead of one (1)?
log_norm_weights = log_weights - log_Z
particle_weights = torch.exp(log_norm_weights).detach().numpy()
assert np.isclose(weights.sum(),1)
particle_idxs = np.random.choice(a=range(n_particles),
                 size=n_particles,
                 p=weights,
                replace=True)

new_particles = []
for idx in range(n_particles):
    new_particles[idx] = particles[particle_idxs[idx]]
    
    
    

/var/folders/bg/cb0cr7ls61352lhy50167r0c0000gn/T/ipykernel_4547/222294140.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_weights = tensor(log_weights)


In [96]:
from smc import run_until_observe_or_end

In [100]:
res = run_until_observe_or_end(res)
cont, args, sigma = res
res

(<evaluator.Procedure at 0x1373912e0>,
 [tensor(8)],
 {'type': 'observe',
  'address': 'addr_start_2_3_4_8',
  'distribution': Normal(loc: -1.3790202140808105, scale: 1.4142135381698608),
  'observed_constant': tensor(8)})

In [107]:
res

(<evaluator.Procedure at 0x1373912e0>,
 [tensor(8)],
 {'type': 'observe',
  'address': 'addr_start_2_3_4_8',
  'distribution': Normal(loc: -1.3790202140808105, scale: 1.4142135381698608),
  'observed_constant': tensor(8)})

In [102]:
weights[i] = sigma['distribution'].log_prob(sigma['observed_constant'])
weights[i]

-23.257019

In [110]:
sum([1,2,3])

6